In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:

from unittest.mock import MagicMock, patch
from src.user.user_management import UserManagement
from src.user.user_model import User
from src.schedule.schedule_model import Schedule
import bcrypt
user_mock = MagicMock()
"""
User have:
 - id
 - username
 - email
 - schedules
 - hashed_password
 - user_preferences
 - get_schedules()
"""

event1 = MagicMock()
event2 = MagicMock()
event3 = MagicMock()
type(event1).title = "event1"
type(event2).title = "event2"
type(event3).title = "event3"

private_schedule = MagicMock()
private_schedule_events = [
    event1, event2, event3
]
# override .elements method

type(user_mock).id = '1'
type(user_mock).username = 'test'
type(user_mock).email = 'test@mail.com'
type(user_mock).schedules = ["private_schedule"]
schedule_dict = {
    "private_schedule": private_schedule
}
salt = bcrypt.gensalt()
type(user_mock).hashed_password = bcrypt.hashpw("test".encode('utf-8'), salt)
type(user_mock).user_preferences = {}
def get_user_schedule_mock(self):
    schedules = self.schedules
    obj_schedules = []
    for schedule in schedules:
        obj_schedules.append(schedule_dict[schedule])
    return obj_schedules
type(user_mock).get_schedules = get_user_schedule_mock
# setting the .events property
type(user_mock).events = private_schedule_events
def get_events_mock(self):
    events = []
    for schedule in self.schedules:
        events += schedule_dict[schedule].get_events()

In [77]:
from src.app.application import Application
import unittest
from src.auth.authentication import AuthenticationModule

from src.app.ui import TkinterUI
# from src.app.states.sign_in_up_state import SignInUp
from src.app.app_states import SignInUp


if __name__ == '__main__':
    db = MagicMock()
    app = Application(db=db)
    # overwrite the authentication of the user \"test\" password \"test\"
    with unittest.mock.patch.object(AuthenticationModule, 'authenticate_user', return_value=True):
        with unittest.mock.patch.object(UserManagement, 'get_user', return_value=user_mock):
            initial_state = SignInUp(app)
            ui = TkinterUI(app)
            app._ui = ui
            app.transition_to(initial_state)
            app.run()
   

Application: Transition to SignInUp
Rendering sign in/up page...
Application: Transition to LoggedOut
Rendering logged out page...
Logging in...
User  authenticated.
Application: Transition to MainState
Rendering main page...
User events: [<MagicMock id='2125863871824'>, <MagicMock id='2125853044944'>, <MagicMock id='2125864203856'>], with len =3


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Cole\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "c:\Users\Cole\OneDrive - Fundacao Getulio Vargas - FGV\Área de Trabalho\Engenharia_Software\src\app\app_states.py", line 46, in login
    self.context.transition_to(MainState(self.context))
  File "c:\Users\Cole\OneDrive - Fundacao Getulio Vargas - FGV\Área de Trabalho\Engenharia_Software\src\app\application.py", line 32, in transition_to
    state.render()
  File "c:\Users\Cole\OneDrive - Fundacao Getulio Vargas - FGV\Área de Trabalho\Engenharia_Software\src\app\app_states.py", line 92, in render
    self.context._ui.user_events.configure(text=event_string)
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'configure'


A janela foi fechada
